# Creating your own pdf

A core feature of zfit is the ability to create custom pdfs and functions in an simple and straightforward way.

There are two main possibilities to create a custom pdf, an easier for most use-cases and an advanced way..

## The simple way

While the same works for functions, an example with a PDF is shown here

In [1]:
import zfit
from zfit import ztf

In [2]:
class MyGauss(zfit.pdf.ZPDF):
    _N_OBS = 1  # dimension, can be omitted
    _PARAMS = ['mean', 'std']  # the name of the parameters
    
    def _unnormalized_pdf(self, x):
        x = ztf.unstack_x(x)  # returns a list with the columns: do x, y, z = ztf.unstack_x(x) for 3D
        mean = self.params['mean']
        std = self.params['std']
        return ztf.exp(- ((x - mean)/std)**2)
        

Done. Now we can use our pdf already!

In [3]:
data_np = np.random.random(size=1000)
data = zfit.data.Data.from_numpy(array=data_np, obs='obs1')

Create two parameters and an instance of your own pdf

In [4]:
mean = zfit.Parameter("mean", 1.)
std = zfit.Parameter("std", 1.)
my_gauss = MyGauss(obs='obs1', mean=mean, std=std)

In [5]:
probs = my_gauss.pdf(data, norm_range=(-3, 4))

In [6]:
probs_np = zfit.run(probs)
print(probs_np[:20])

[0.3415578  0.32765783 0.52652506 0.34822012 0.48106853 0.47882796
 0.56385067 0.5467811  0.48493075 0.40819233 0.54168993 0.51195988
 0.47842369 0.52624962 0.2567314  0.25996271 0.50516375 0.54637628
 0.46770326 0.40367536]


We could improve our PDF by registering an integral

In [7]:
def gauss_integral_from_any_to_any(limits, params, model):
    (lower,), (upper,) = limits.limits
    mean = params['mean']
    std = params['std']
    # write your integral here
    return 42.  # dummy integral

In [1]:
limits = zfit.Space.from_axes(axes=0, limits=(zfit.Space.ANY_LOWER, zfit.Space.ANY_UPPER))
MyGauss.register_analytic_integral(func=gauss_integral_from_any_to_any, limits=limits)

NameError: name 'MyGauss' is not defined

### n-dimensional

## Advanced Custom PDF

Subclass BasePDF. The `_unnormalized_pdf` has to be overriden and, in addition, the `__init__`.

Any of the public main methods (`pdf`, `integrate`, `partial_integrate` etc.) can **always** be overriden by implementing the function with a leading underscore, e.g. implement `_pdf` to directly controls `pdf`, the API is the same as the public function without the name. In case, during execution of your own method, it is found to be a bad idea to have overridden the default methods, throwing a `NotImplementedError` will restore the default behavior.

In [1]:
# TOBEDONE